<a href="https://colab.research.google.com/github/Vise-1/Eye_Disease_Classification_via_ML/blob/main/Model_Eye_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Dependencies.
import requests
import zipfile
from pathlib import Path
from tqdm.auto import tqdm
import shutil
import os

import torch
from torch import nn

# Device agnostics.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
url = "https://github.com/PerceptiLabs/ocular-disease/archive/refs/heads/main.zip"

# Creating directory and Downloading data
data_path = Path("data/")
img_path = data_path / "Dataset_Eye_Disease"

if img_path.is_dir():
  print(f"{img_path} directory already exists..... SKIPPING DOWNLOAD")
else:
  print(f"{img_path} does not exists, creating one...")
  img_path.mkdir(parents=True, exist_ok=True)

# Download dataset from github (same as kaggle but calling the API was a hassle so opted for this PS: Much easier this way, trust me)
with open(data_path / "ocular-disease-main.zip", "wb") as f:
  request = requests.get(url)
  print("Downloading Eye Disease Dataset...")
  f.write(request.content)

# Unzipping data to created folder
with zipfile.ZipFile(data_path / "ocular-disease-main.zip", "r") as zip_ref:
  print(f"Unzipping Eye Disease Dataset...")
  zip_ref.extractall(img_path)

data/Dataset_Eye_Disease directory already exists..... SKIPPING DOWNLOAD
Unzipping Eye Disease Dataset...
